# Initialization

In [ ]:
import sys
import tensorflow as tf
import pandas as pd
import numpy as np
from joblib import dump, load
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.layers import Dense

print (sys.version)
print(tf.__version__)
if tf.test.gpu_device_name(): 
    print(tf.test.is_gpu_available())
    print(tf.test.gpu_device_name())
else:
   print("Please install GPU version of TF")

In [ ]:
df = pd.read_csv("result.csv")
input_dim = 95 + 32 + 211

bitsColsX = [ 'x{}'.format(i) for i in range(95) ]
bitsColsR = [ 'r{}'.format(i) for i in range(32) ]
bitsColsP = [ 'p{}'.format(i) for i in range(211) ]
X = df.loc[:, bitsColsX + bitsColsR + bitsColsP]
Y = df.loc[:, 'Class']
#15% train 85% test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.85, random_state=0)

#We initialize data structures and the regularization parameter.
regul_param = 10.0 ** -2

train_scores = []
test_scores = []
classifiers = []
clf_name = []
clf_topology = []

# Classifier Definitions

## 100 Neuron Classifiers

In [ ]:
clf100 = Sequential()
clf100.add(Dense(100, input_dim=input_dim, activation='relu'))
clf100.add(Dense(1, activation='sigmoid'))

clf100.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf100)
clf_name.append("clf100")
clf_topology.append("100")

In [ ]:
clf5050 = Sequential()
clf5050.add(Dense(50, input_dim=input_dim, activation='relu'))
clf5050.add(Dense(50, activation='relu'))
clf5050.add(Dense(1, activation='sigmoid'))

clf5050.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf5050)
clf_name.append("clf5050")
clf_topology.append("50-50")

In [ ]:
clf80 = Sequential()
clf80.add(Dense(80, input_dim=input_dim, activation='relu'))
clf80.add(Dense(15, activation='relu'))
clf80.add(Dense(5, activation='relu'))
clf80.add(Dense(1, activation='sigmoid'))

clf80.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf80)
clf_name.append("clf80")
clf_topology.append("80-15-5")

In [ ]:
clf70 = Sequential()
clf70.add(Dense(70, input_dim=input_dim, activation='relu'))
clf70.add(Dense(25, activation='relu'))
clf70.add(Dense(5, activation='relu'))
clf70.add(Dense(1, activation='sigmoid'))

clf70.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf70)
clf_name.append("clf70")
clf_topology.append("70-25-5")

In [ ]:
clf525 = Sequential()
clf525.add(Dense(50, input_dim=input_dim, activation='relu'))
clf525.add(Dense(25, activation='relu'))
clf525.add(Dense(25, activation='relu'))
clf525.add(Dense(1, activation='sigmoid'))

clf525.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf525)
clf_name.append("clf525")
clf_topology.append("50-25-25")

In [ ]:
clf530 = Sequential()
clf530.add(Dense(50, input_dim=input_dim, activation='relu'))
clf530.add(Dense(30, activation='relu'))
clf530.add(Dense(10, activation='relu'))
clf530.add(Dense(10, activation='relu'))
clf530.add(Dense(1, activation='sigmoid'))

clf530.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf530)
clf_name.append("clf530")
clf_topology.append("50-30-10-10")

## 200 Neuron Classifiers

In [ ]:
clf200 = Sequential()
clf200.add(Dense(200, input_dim=input_dim, activation='relu'))
clf200.add(Dense(1, activation='sigmoid'))

clf200.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf200)
clf_name.append("clf200")
clf_topology.append("200")

In [ ]:
clf155 = Sequential()
clf155.add(Dense(100, input_dim=input_dim, activation='relu'))
clf155.add(Dense(50, activation='relu'))
clf155.add(Dense(50, activation='relu'))
clf155.add(Dense(1, activation='sigmoid'))

clf155.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf155)
clf_name.append("clf155")
clf_topology.append("100-50-50")

In [ ]:
clf160 = Sequential()
clf160.add(Dense(160, input_dim=input_dim, activation='relu'))
clf160.add(Dense(25, activation='relu'))
clf160.add(Dense(10, activation='relu'))
clf160.add(Dense(5, activation='relu'))
clf160.add(Dense(1, activation='sigmoid'))

clf160.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf160)
clf_name.append("clf160")
clf_topology.append("160-25-10-5")

# Training & Testing

In [ ]:
%%time
clfnum = 0
for clf in classifiers:
    train = []
    test = []
    runs = 10
    for i in range(runs):
        print("Classifier: ", (clfnum +1), " Run: ", (i+1))
        clfaux= tf.keras.models.clone_model(clf)
        clfaux.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        clfaux.fit(X_train, y_train, epochs=300, batch_size=16384,verbose = 0)
        train.append(clfaux.evaluate(X_train, y_train, verbose = 0)[1])
        test.append(clfaux.evaluate(X_test, y_test, verbose = 0)[1])
        filename = 'trained_models/'+clf_name[clfnum] +'_v' + str(i+1) +'.joblib'
        clfaux.save(filename)
    clfnum += 1    
    train_scores.append(train)
    test_scores.append(test)

In [ ]:
print(clf_name)
print(clf_topology)
print(train_scores)
print(test_scores)

# Results Visualization

In [ ]:
train = []
test = []
train_err = []
test_err = []

for i in range(len(clf_name)):
    train.append(np.mean(train_scores[i]))
    train_err.append(np.std(train_scores[i]))
    test.append(np.mean(test_scores[i]))
    test_err.append(np.std(test_scores[i]))
means = pd.DataFrame({'Train': train,'Test': test}, index=clf_topology)
errors = pd.DataFrame({'Train': train_err,'Test': test_err}, index=clf_topology)


In [ ]:
means

In [ ]:
errors

In [ ]:
means = pd.DataFrame({'Train': train,'Test': test}, index=clf_topology)
errors = pd.DataFrame({'Train': train_err,'Test': test_err}, index=clf_topology)

sup = 1.005*max(max(train),max(test))
inf = 0.995*min(min(train),min(test))
ax = means.plx = means.plot.bar(rot=0,yerr=errors,figsize=(12,10),ylim=[inf,sup],capsize=4, xlabel= "Topology", ylabel= "Accuracy")